In [17]:
import torch
from torch.autograd import Variable

import lxmls.readers.sentiment_reader as srs
from lxmls.deep_learning.utils import AmazonData, Model, glorot_weight_init, index2onehot
corpus=srs.SentimentCorpus("books")
data = AmazonData(corpus=corpus)

if (torch.cuda.is_available):
    device_type = "cuda"
    device = torch.device(device_type)

In [22]:
class PytorchLogLinear(Model):
    def __init__(self):
        # Initialize parameters
        weight_shape = (config['input_size'], config['num_classes'])
        self.weight = glorot_weight_init(weight_shape, 'softmax')
        self.bias = np.zeros((1, config['num_classes']))
        self.learning_rate = config['learning_rate']
    
        self.weight = Variable(torch.from_numpy(self.weight, device=device).float(), requires_grad=True)
        self.bias = Variable(torch.from_numpy(self.bias, device=device).float(), requires_grad=True)
    
        self.logsoftmax = torch.nn.LogSoftmax(dim=1, device=device)
        self.loss = torch.nn.NLLLoss(device=device)
    
    def _log_forward(self, input=None):
        input = Variable(torch.from_numpy(input).float(), requires_grad=False)
        z = torch.matmul(input, torch.t(self.weight)) + self.bias
        
        log_tilde_z = self.logsoftmax(z)
        
        return log_tilde_z
    
    def predict(self, input=None):
        log_forward = self._log_forward(input).data.numpy()
        return np.argmax(np.exp(log_forward), axis=1)
    
    def update(self, input=None, output=None):
        """Stochastic Gradient Descent update"""
        true_class = Variable(torch.from_numpy(output).long(), requires_grad=False)
        
        loss = self.loss(self._log_forward(input), true_class)
        loss.backwards()
        
        self.weight.data -= self.learning_rate * self.weight.grad.data
        self.weight.grad.data.zero_()
        slef.bias.data -= self.learning_rate * self.bias.grad.data
        self.bias.grad.data.zero_()
               
        return loss.data.numpy()

In [23]:
# Instantiate model
model = PytorchLogLinear(
    input_size=corpus.nr_features,
    num_classes=2,
    learning_rate=0.05
)

TypeError: __init__() got an unexpected keyword argument 'input_size'